In [39]:
import struct, socket
import numpy as np
import linecache, bisect
import csv
import operator

contxtpath = '/home/hadoop/ipython/vast/iploc/'
iploc = contxtpath + 'iploc.csv'
nwloc = contxtpath + 'networkcontext.txt'

spath = '/home/hadoop/ipython/vast/user/vast/'
#sconnect = spath + 'lda_scores_20140708.csv'
#sconnectbu = spath + 'lda_scores_20140708_bu.csv'

In [43]:
#set anchor IP (focus of investigation)
# summary query for all IP addresses on all ports that connected with anchor IP
anchor_ip = '10.9.81.5'
ir_f = spath + "ir-" + anchor_ip.replace(".","_") + ".tsv"
dy ='6'
hivestr =  ("\"set hive.cli.print.header=true;SELECT " + 
"min(parseddate) as firstSeen, max(parseddate) as lastSeen, " + 
"firstseensrcip as srcIP, firstseendestip as dstIP, " + 
"firstseensrcport as SPort, firstseensrcport AS Dport, " +
"count(firstseensrcip) as conns, max(firstseendestpacketcount) as maxPkts, " +
"avg(firstseendestpacketcount) as avgPkts, " +
"max(firstseendestpayloadbytes) as maxBytes, " +
"avg(firstseendestpayloadbytes) as avgBytes " +
"FROM vast_netflow WHERE day(parseddate)=" +dy+" AND " + 
"(firstseensrcip =\'" + anchor_ip + "\'  OR firstseendestip=\'" + anchor_ip + "\') " +
"GROUP BY firstseensrcip, firstseendestip,firstseensrcport,firstseendestport\"  > " + ir_f)
print hivestr
!hive -S -e $hivestr


"set hive.cli.print.header=true;SELECT min(parseddate) as firstSeen, max(parseddate) as lastSeen, firstseensrcip as srcIP, firstseendestip as dstIP, firstseensrcport as SPort, firstseensrcport AS Dport, count(firstseensrcip) as conns, max(firstseendestpacketcount) as maxPkts, avg(firstseendestpacketcount) as avgPkts, max(firstseendestpayloadbytes) as maxBytes, avg(firstseendestpayloadbytes) as avgBytes FROM vast_netflow WHERE (firstseensrcip ='10.9.81.5'  OR firstseendestip='10.9.81.5') GROUP BY firstseensrcip, firstseendestip,firstseensrcport,firstseendestport"  > /home/hadoop/ipython/vast/user/vast/ir-10_9_81_5.tsv
15/04/21 23:12:56 WARN conf.HiveConf: DEPRECATED: Configuration property hive.metastore.local no longer has any effect. Make sure to provide a valid value for hive.metastore.uris if you are connecting to a remote metastore.


In [44]:
# calculate number of inbound only, two-way, and outbound only
# build dict of IP addresses
#firstSeen,lastSeen,srcIP, dstIP, sport,dport,conns, maxPkts, avgPkts,maxBytes, avgBytes
srcdict = {}
dstdict = {}
with open(ir_f, 'r') as f:
    reader = csv.reader(f,delimiter='\t') 
    reader.next();
    rowct = 1
    for row in reader:
        srcdict[row[2]] = struct.unpack("!L", socket.inet_aton(row[2]))[0]
        dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
        rowct += 1
inbound = {};
outbound = {};
twoway = {};
for result in srcdict:
    if result in dstdict:
        twoway[result] = srcdict[result]
    else:
        outbound[result] = srcdict[result]
for result in dstdict:
    
    if result not in srcdict:
        inbound[result] = dstdict[result]  
   

In [52]:
print outbound.keys()
print twoway.keys()
print len(inbound)
print inbound.keys()

[]
['10.9.81.5', '172.10.0.40', '172.30.0.4', '172.10.0.3', '172.30.0.6']
11
['172.30.0.7', '172.30.0.5', '172.10.0.9', '172.10.0.8', '172.20.0.6', '172.10.0.5', '172.10.0.4', '172.10.0.7', '172.10.0.2', '172.30.0.3', '172.30.0.2']


In [18]:
# load IP location information -- LONG step
# load fromIP into ndarray
iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})
# get geospatial info
for srcip in outbound:
    outbound[srcip] = linecache.getline(iploc, bisect.bisect(iplist,outbound[srcip]))
print "outbound connections located."
for dstip in twoway:
        twoway[dstip] = linecache.getline(iploc, bisect.bisect(iplist,twoway[dstip]))
print "twoway connections located."
for srcip in inbound:
        inbound[srcip] = linecache.getline(iploc, bisect.bisect(iplist,inbound[srcip]))
print "inbound connections located."
#print outbound
#print twoway

IOError: [Errno 2] No such file or directory: '/home/hadoop/ipython/vast/iploc/iploc.csv'

In [123]:
print inbound

{'146.185.239.104': '"2461658880","2461659135","LU","Luxembourg","Luxembourg","Luxembourg","49.611670","6.130000","Cubehost","-"\n'}


In [19]:
# need some way to combine timelines of outbound and two-way with big picture inbound only

# get network context - get start and end ranges
## need to write to updated file
with open(nwloc, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    #reader.next()
    nwdict = {}
    rowct = 1
    #address range, description
    for row in reader:
        iprange = row[0].split('/')
        if len(iprange) < 2:
            ipend = 0
        else:
            ipend = int(iprange[1])
        nwdict[row[0]] = [struct.unpack("!L", socket.inet_aton(iprange[0]))[0],
                          struct.unpack("!L", socket.inet_aton(iprange[0]))[0]+2**(32-ipend)-1,
                          row[1]]
        #dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
        rowct += 1
 
# get geospatial info
nwoutbound = {}
nwtwoway = {}
nwinbound = {}
for srcip in outbound:
        nwoutbound[srcip] = linecache.getline(nwloc, bisect.bisect(iplist,outbound[srcip]))
print "outbound connections located."
for dstip in twoway:
        nwtwoway[dstip] = linecache.getline(nwloc, bisect.bisect(iplist,twoway[dstip]))
print "twoway connections located."
for srcip in inbound:
        nwinbound[srcip] = linecache.getline(nwloc, bisect.bisect(iplist,inbound[srcip]))
print "inbound connections located."



outbound connections located.


NameError: name 'iplist' is not defined

In [ ]:
print nwoutbound

In [ ]:
# how many bytes were exchanged

# detail query for two-way and outbound only
"set hive.cli.print.header=true;SELECT tstart, tend, sip as srcIP, 
dip as dstIP, proto as Proto, sport as SPort, dport AS Dport,ipkt as 
Pkts, ibyt as Bytes  FROM netflow_20140708 WHERE 
(dip IN('213.190.153.47','10.128.30.62','134.191.242.7','134.191.236.133',
        '174.139.113.58','134.191.236.140','134.191.240.76','134.191.240.74',
        '134.191.241.36','134.191.240.112', '134.191.240.68','134.191.240.106')
 AND sip ='174.139.113.58') OR 
(sip IN('213.190.153.47','10.128.30.62','134.191.242.7','134.191.236.133',
        '174.139.113.58','134.191.236.140','134.191.240.76','134.191.240.74',
        '134.191.241.36','134.191.240.112', '134.191.240.68','134.191.240.106') 
 AND dip ='174.139.113.58') SORT BY tstart"  > /home/hadoop/ipython/user/gbabb/sbdet-174_139_113_58.tsv

<h4> Save for Storyboard </h4>

In [ ]:
# push to json  worldmap
for srcip in twoway:
    row =  twoway[srcip].replace('"','').split(',')
    #print row  #row[7],row[6],
    elem = ("{ \"type\": \"Feature\",\"properties\":{\"location\":\"" + row[8] + "\"" +
    ",\"ip\":\""+srcip+"\",\"type\":1}, \"geometry\": { \"type\": \"Point\", \"coordinates\": " +
    "["+row[7]+","+ row[6]+" ] } }\n,")
    #print elem
rowct = 1
for srcip in inbound:
    #print inbound[srcip]
    row =  inbound[srcip].replace('"','').split(',')
    #create hash of iploc and only print unique ones
    if len(row) > 8 and (rowct>=100001 and rowct <= 103000):
        elem = ("{ \"type\": \"Feature\",\"properties\":{\"location\":\"" + row[8] + "\"" +
        ",\"ip\":\""+srcip+"\",\"type\":2}, \"geometry\": { \"type\": \"Point\", \"coordinates\": " +
        "["+row[7]+","+ row[6]+" ] } }\n,")
        print elem
    rowct += 1
    


In [ ]:
# record analysis
# push to json file(s) - file names for event timeline, worldmap, dendrogram


In [55]:
#timeline query
anchor_ip = '10.9.81.5'
dy = '14'
sb_f = spath + "sbdet-" + anchor_ip.replace(".","_") + ".tsv"

hivestr =  ("\"set hive.cli.print.header=true;SELECT " + 
"parseddate as firstseen, firstseensrcip as srcip, firstseendestip as dstip, " + 
"firstseensrcport as sport, firstseendestport AS dport, " +
"firstseendestpacketcount as pkts, firstseendestpayloadbytes as bytes " +
"FROM vast_netflow WHERE "#day(parseddate)=" +dy+" AND " + 
"(firstseensrcip =\'" + anchor_ip + "\'  OR firstseendestip=\'" + anchor_ip + "\')" + 
"  LIMIT 50000  \"  > " + sb_f)
print hivestr
!hive -S -e $hivestr

"set hive.cli.print.header=true;SELECT parseddate as firstseen, firstseensrcip as srcip, firstseendestip as dstip, firstseensrcport as sport, firstseendestport AS dport, firstseendestpacketcount as pkts, firstseendestpayloadbytes as bytes FROM vast_netflow WHERE (firstseensrcip ='10.9.81.5'  OR firstseendestip='10.9.81.5')  LIMIT 50000  "  > /home/hadoop/ipython/vast/user/vast/sbdet-10_9_81_5.tsv
15/04/21 23:29:17 WARN conf.HiveConf: DEPRECATED: Configuration property hive.metastore.local no longer has any effect. Make sure to provide a valid value for hive.metastore.uris if you are connecting to a remote metastore.


In [53]:
!echo $ir_f
!wc -l $sb_f
!head -n 10 $sb_f

/home/hadoop/ipython/vast/user/vast/ir-10_9_81_5.tsv
150001 /home/hadoop/ipython/vast/user/vast/sbdet-10_9_81_5.tsv
firstseen	srcip	dstip	sport	dport	pkts	bytes
2013-04-06 11:19:46	10.9.81.5	172.30.0.3	51358	58513	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.2	51358	7740	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.3	51358	54923	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.2	51358	46183	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.3	51358	30608	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.2	51358	17046	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.3	51358	49412	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.2	51358	27451	0	0
2013-04-06 11:19:46	10.9.81.5	172.30.0.3	51358	55947	0	0
